In [2]:
import json
from summarizer import Summarizer

In [3]:
args = [json.loads(ln)["argument"]["argument"] for ln in open("../data/cmv_processed.jsonl")]
ids = [json.loads(ln)["id"] for ln in open("../data/cmv_processed.jsonl")]

In [4]:
corpus = set()
idx = set()

for j, k in zip(args, ids):
    corpus.add((j, k))

corpus = list(corpus)
type(corpus), len(corpus)

(list, 5990)

In [5]:
model = Summarizer()

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [27]:
from tqdm.notebook import tqdm
results = []

with tqdm(total=(len(corpus)), position=0, leave=True) as pbar:
    for i, j in corpus:
        results.append((j, model(i, num_sentences = 3, min_length=10)))
        pbar.update()

  0%|          | 0/10303 [00:00<?, ?it/s]

In [29]:
len(results)

5990

In [1]:
len(corpus)

NameError: name 'corpus' is not defined

In [31]:
fout = open("../data/argument_extracts.jsonl", "w")
for j, k in results:
    fout.write(json.dumps({
        "id": j,
        "extract": k,
    }))
    fout.write("\n")

In [4]:
from collections import defaultdict
from tqdm.notebook import tqdm
import os
from src.utils_.utils import sentences_segment, tokeniser
import sys

os.chdir("/Users/joshua.sheppard/PycharmProjects/countaBot")
print(os.getcwd())

from BERT_adu_classifier import predict

sample = corpus
adus = {}
for i, j in corpus:
    adus[j] = {"argument": [], "label": []}

adu_lst = []

# Detect Opinionated Sentences; Yield focused arguments
with tqdm(total=len(sample), position=0, leave=True) as pbar:
    for arg, idx in sample:
        adu_lst = []
        for sent in sentences_segment(arg):
            # if predict(sent) == "claim":
            #     adu_lst.append((sent, "claim"))

            if predict(sent) == "premise":
                adu_lst.append((sent, "premise"))-

            else: continue

        adus[idx] = {"argument": adu_lst, "label": "premise"}

        pbar.update()

/Users/joshua.sheppard/PycharmProjects/countaBot
claim
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  0%|          | 0/5990 [00:00<?, ?it/s]

In [81]:
adus

{'t3_2daybl': {'argument': [('Everyone deals with the lingering thoughts of suicide or has in the past.',
    'premise'),
   ('No one is immune to these thoughts.', 'premise'),
   ('Many of you reading this right now deal with severe forms of depression bipolar schizophrenia addiction you name it.',
    'premise'),
   ('And guess what?', 'premise'),
   ('YOURE ALIVE!', 'premise')],
  'label': 'premise'},
 't3_6udz7t': {'argument': [('Ill focus on the US for this because I have readily available data.I often hear the argument that we should legalize marijuana because it is less harmful than alcohol a substance already legal in the US.',
    'premise'),
   ('In people died in alcoholimpaired driving crashes accounting for nearly onethird of all traffic related deaths in the US This is before we even account for alcohol provoked fights domestic violence etc.Second alcohol does little good for our society.',
    'premise'),
   ('Heres a NY Times article from that ultimately shines a positi

In [ ]:
fout = open("../data/argument_premises.jsonl", "w")
for j, k in adus:
    fout.write(json.dumps({
        "id": j,
        "premises": k,
    }))
    fout.write("\n")